Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [1]:
library('tidymodels')
library('ISLR2')



── Attaching packages ────────────────────────────────────── tidymodels 1.2.0 ──

✔ broom        1.0.5      ✔ recipes      1.0.10
✔ dials        1.2.1      ✔ rsample      1.2.1 
✔ dplyr        1.1.4      ✔ tibble       3.2.1 
✔ ggplot2      3.5.1      ✔ tidyr        1.3.1 
✔ infer        1.0.7      ✔ tune         1.2.1 
✔ modeldata    1.3.0      ✔ workflows    1.1.4 
✔ parsnip      1.2.1      ✔ workflowsets 1.1.0 
✔ purrr        1.0.2      ✔ yardstick    1.3.1 

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()
• Search for functions across packages at https://www.tidymodels.org/find/



In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [3]:
?Default

Default {ISLR2},R Documentation


In [2]:
Default |> head()

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [4]:
Default |> count(default)

default,n
<fct>,<int>
No,9667
Yes,333


The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [5]:
Default_split = initial_split(Default, prop = 0.90, strata = default)

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [6]:
mod = logistic_reg() |>
set_engine("glm") |>
set_mode("classification")



Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [7]:
# This code calculates the accuracy of a model that always predicts default to be "No"

Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred_naive)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9677778


Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [9]:
folds = vfold_cv(Default, v = 10, strata = default)

folds |> glimpse()


Rows: 10
Columns: 2
$ splits <list> [<vfold_split[9000 x 1000 x 10000 x 4]>], [<vfold_split[9000 x…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [10]:
mod |> 
    fit_resamples(default ~ income + balance, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97380000,10,0.002069890,Preprocessor1_Model1
brier_class,binary,0.02147391,10,0.001632933,Preprocessor1_Model1
roc_auc,binary,0.95039946,10,0.004126309,Preprocessor1_Model1


❓How does the model accuracy compare to the naive model from above?

The logistic regression model's accuracy has a mean value of 0.9738 (97.38% accuracy) with a standard error of 0.002069890 (i.e. 0.21%). It also implies that the model's accuracy is within a 0.9697 - 0.9779 range (using 95% confidence interval range):
0.9697 = 0.9738 - (1.96 * 0.002069890); 
 0.9779 = 0.9738 + (1.96 * 0.002069890)

The 0.9738 mean accuracy value of the model (as well as the lower and the higher end of its 95% confidence interval range) is above the 0.9677778 accuracy of the naive model.

Thus the logistic regression model's accuracy is clearly higher than the naive model's accuracy (although, by not much).

In addition, I think that a higher rate of "Yes" enties in the "default" variable would make the accuracy of the naive model lower, but would not necessarily make the accuracy of the logistic regression model lower. Thus I would expect a larger difference between the logistic regression model's accuracy and the naive model's accuracy in such situation (the logistic regression model's accuracy being higher compared to the naive model's accuracy) and vice versa.

Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [15]:
rec = recipe(default ~ income + balance + student, data = Default) |>
    step_dummy(student)

mod |>
    fit_resamples(rec, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97320000,10,0.001965254,Preprocessor1_Model1
brier_class,binary,0.02139113,10,0.001600257,Preprocessor1_Model1
roc_auc,binary,0.95078181,10,0.003991393,Preprocessor1_Model1


❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?

No, by adding student as part of the explanatory variables actually decreased the mean accuracy of the logistic regression model from 0.9738 to 0.9732.



Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

❓Does our model outperform a naive model?

In [20]:
folds2 = vfold_cv(Default_test, v = 10, strata = default)

mod |> 
    fit_resamples(default ~ income + balance, folds2) |>
    collect_metrics()


.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.96800000,10,0.003887301,Preprocessor1_Model1
brier_class,binary,0.02463054,10,0.003255276,Preprocessor1_Model1
roc_auc,binary,0.96831308,10,0.010219083,Preprocessor1_Model1


Yes, the logistic regression model run on the Default_test data had a mean accuracy of 0.968, which slightly outperformed the naive model (which had an accuracy level of 0.9677778).

And again, I think that a higher rate of "Yes" enties in the "default" variable would make the accuracy of the naive model lower, but would not necessarily make the accuracy of the logistic regression model lower. Thus I would expect a larger difference between the logistic regression model's accuracy and the naive model's accuracy in such situation (the logistic regression model's accuracy being higher compared to the naive model's accuracy) and vice versa.